In [75]:
import numpy as np
from StringIO import StringIO
import string

In [76]:
data = np.genfromtxt('movie_metadata.csv', dtype=None, delimiter=',', autostrip=True)
colNames = data[0,:]
movieData = np.delete(data, (0), axis=0)
numMovies = movieData.shape[0]
print numMovies 

5041


In [77]:
genres = {'Action', 'Adventure','Animation','Biography','Comedy','Crime','Documentary','Drama','Family','Fantasy',
          'Film-Noir','Game-Show','History','Horror','Music','Musical','Mystery','News','Reality-TV','Romance','Sci-Fi',
          'Short','Sport','Thriller','War','Western'}

moviesByGenreBucket = {}

for genre in genres:
    moviesByGenreBucket[genre] = []
    

In [78]:
for movie in movieData:
    localGenres = string.split(movie[10], "|")
    for genre in localGenres:
        #print "adding movieID = %s to genre bucket %s" % (movie[0], genre)
        moviesByGenreBucket[genre].append(movie[0])

KeyError: 'CCH Pounder'

In [79]:
for key in moviesByGenreBucket.keys():
    print "%s = %s" % (key, len(moviesByGenreBucket[key]))


Sci-Fi = 0
Crime = 0
Romance = 0
Animation = 0
Music = 0
Comedy = 0
War = 0
Horror = 0
Film-Noir = 0
Western = 0
News = 0
Reality-TV = 0
Thriller = 0
Adventure = 0
Mystery = 0
Short = 0
Drama = 0
Action = 0
Documentary = 0
Musical = 0
History = 0
Family = 0
Fantasy = 0
Game-Show = 0
Sport = 0
Biography = 0


In [80]:
print moviesByGenreBucket['Game-Show']

[]


In [81]:
movieData[2143]

array(['Color', '', '5', '60', '', '', '', '98', '',
       'Game-Show|Reality-TV|Romance', 'Chris Harrison',
       '"""The Bachelor            """', '4398', '98', '', '0',
       'bachelor|seeking love|single guy|tv host|women rivals for man',
       'http://www.imdb.com/title/tt0313038/?ref_=fn_tt_tt_1', '33',
       'English', 'USA', '', '3000000', '', '', '2.9', '', '141'], 
      dtype='|S149')